# Crema-D TRILL Extraction - Morgan Sandler (sandle20@msu.edu)

In [1]:
import librosa
import numpy as np
import pandas as pd
import glob
import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()
assert tf.executing_eagerly()

import tensorflow_hub as hub
import numpy as np

wav_files = glob.glob('../../data/AudioWAV/*')

# filepath, speaker, sentence, emotion, intensity
data_raw = {
    'filepath' : [],
    'speaker' : [],
    'sentence' : [],
    'emotion' : [],
    'intensity' : [],
    'waveform' : []
}

for wav_file in wav_files:
    speaker, sentence, emotion, intensity = wav_file.split('/')[-1].split('.')[0].split('_')
    data_raw['filepath'].append(wav_file)
    data_raw['speaker'].append(speaker)
    data_raw['sentence'].append(sentence)
    data_raw['emotion'].append(emotion)
    data_raw['intensity'].append(intensity)
    
    wav_data, sr = librosa.load(wav_file, sr=16000, mono=True)
    data_raw['waveform'].append(wav_data)

data_df = pd.DataFrame(data_raw)
print(data_df.head(10))

2022-10-04 13:25:13.770084: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


                                  filepath speaker sentence emotion intensity  \
0  ../../data/AudioWAV/1022_ITS_ANG_XX.wav    1022      ITS     ANG        XX   
1  ../../data/AudioWAV/1037_ITS_ANG_XX.wav    1037      ITS     ANG        XX   
2  ../../data/AudioWAV/1060_ITS_NEU_XX.wav    1060      ITS     NEU        XX   
3  ../../data/AudioWAV/1075_ITS_NEU_XX.wav    1075      ITS     NEU        XX   
4  ../../data/AudioWAV/1073_IOM_DIS_XX.wav    1073      IOM     DIS        XX   
5  ../../data/AudioWAV/1066_IOM_DIS_XX.wav    1066      IOM     DIS        XX   
6  ../../data/AudioWAV/1078_IWL_SAD_XX.wav    1078      IWL     SAD        XX   
7  ../../data/AudioWAV/1029_TAI_FEA_XX.wav    1029      TAI     FEA        XX   
8  ../../data/AudioWAV/1039_IEO_SAD_MD.wav    1039      IEO     SAD        MD   
9  ../../data/AudioWAV/1008_TAI_HAP_XX.wav    1008      TAI     HAP        XX   

                                            waveform  
0  [0.00024414062, -0.0012512207, -0.002532959, -... 

In [ ]:
print(len(data_df))

7442


In [4]:
#extract trills
import tqdm

def get_all_trills():
    all_embeds = []
    module = hub.load('https://tfhub.dev/google/nonsemantic-speech-benchmark/trill/2')

    for wav_data in tqdm.tqdm(data_df['waveform']):
        emb_dict = module(samples=wav_data, sample_rate=16000)
        # For a description of the difference between the two endpoints, please see our
        # paper (https://arxiv.org/abs/2002.12764), section "Neural Network Layer".
        emb = emb_dict['embedding']
        #emb_layer19 = emb_dict['layer19']
        # Embeddings are a [time, feature_dim] Tensors.
        emb.shape.assert_is_compatible_with([None, 512])
        all_embeds.append(emb)
    return all_embeds
        

In [5]:
feats = get_all_trills()

  1%|          | 81/7442 [00:32<49:23,  2.48it/s]  


KeyboardInterrupt: 